In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("spark://192.168.2.97:7077") \
        .appName("Group-6-Question-1-Claude")\
        .config("spark.dynamicAllocation.enabled", True)\
        .config("spark.dynamicAllocation.shuffleTracking.enabled",True)\
        .config("spark.cores.max", 2)\
        .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/03/10 13:23:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/03/10 13:23:08 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/03/10 13:23:09 WARN ExecutorAllocationManager: Dynamic allocation without a shuffle service is an experimental feature.


In [18]:
from pyspark.sql.functions import trim, regexp_replace, lower, explode, trim, split
from pyspark.sql.types import StructType, StructField, StringType

In [14]:
df = spark.read.json("../RC_2006-01.json")
df.show(10)

+--------------------+----------+
|                body| subreddit|
+--------------------+----------+
|early 2006 a prob...|reddit.com|
|If you are going ...|reddit.com|
|Microsoft hates i...|reddit.com|
|this looks intere...|reddit.com|
|I have nothing bu...|reddit.com|
|And by "Microsoft...|reddit.com|
|This is exactly t...|reddit.com|
|           [deleted]|reddit.com|
|Because this one ...|reddit.com|
|                  ok|reddit.com|
+--------------------+----------+
only showing top 10 rows



In [19]:
my_df_lower = df.select(lower("body").alias("lowercase_body"))

my_df_words = my_df_lower.select(explode(split("lowercase_body", "\s+")).alias("word"))

word_counts = my_df_words.groupBy("word").count().orderBy("count", ascending=False)

In [20]:
word_counts.show(1000)

+-------------+-----+
|         word|count|
+-------------+-----+
|          the| 6362|
|           to| 3838|
|            a| 3385|
|           of| 3175|
|          and| 2683|
|           is| 2641|
|         that| 2328|
|            i| 2277|
|           in| 1978|
|           it| 1580|
|          you| 1364|
|          for| 1284|
|         this| 1118|
|          are|  969|
|           be|  947|
|          not|  937|
|          but|  915|
|           on|  894|
|         have|  863|
|           as|  805|
|         with|  798|
|           if|  674|
|         they|  655|
|           or|  599|
|        about|  575|
|          was|  570|
|         just|  535|
|         like|  526|
|           at|  526|
|           an|  523|
|         what|  519|
|       people|  509|
|           so|  496|
|    [deleted]|  471|
|         it's|  469|
|          can|  462|
|         from|  460|
|        would|  457|
|         more|  452|
|           by|  450|
|           he|  411|
|        think|  411|
|         

In [ ]:
spark_session.stop()